## «Модель прогнозирования стоимости жилья для агентства недвижимости»



# Дипломный проект

Data Science

Описание данных:

'status' — статус продажи;
'private pool' и 'PrivatePool' — наличие собственного бассейна;
'propertyType' — тип объекта недвижимости;
'street' — адрес объекта;
'baths' — количество ванных комнат;
'homeFacts' — сведения о строительстве объекта (содержит несколько
типов сведений, влияющих на оценку объекта);

'fireplace' — наличие камина;
'city' — город;
'schools' — сведения о школах в районе;
'sqft' — площадь в футах;
'zipcode' — почтовый индекс;
'beds' — количество спален;
'state' — штат;
'stories' — количество этажей;
'mls-id' и 'MlsId' — идентификатор MLS (Multiple Listing Service, система
мультилистинга);

target' — цена объекта недвижимости (целевой признак, который
необходимо спрогнозировать).

In [152]:
import numpy as np
import pandas as pd
import re
from geopy.geocoders import Nominatim
from fake_useragent import UserAgent
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
from geopy.exc import GeocoderServiceError

Предварительный просмотр

In [153]:
df = pd.read_csv('data/data.csv')
display(df.head())
df.info()

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103115 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226470 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

Пропуски

In [154]:
missing_data = df.isnull().sum()

In [155]:
for column_name, missing_count in missing_data.items():
    if missing_count > 0:
        percentage = (missing_count / len(df)) * 100
        print(f'{column_name}: {missing_count} пропусков ({percentage:.2f}%)')

status: 39918 пропусков (10.58%)
private pool: 373004 пропусков (98.89%)
propertyType: 34733 пропусков (9.21%)
street: 2 пропусков (0.00%)
baths: 106338 пропусков (28.19%)
fireplace: 274070 пропусков (72.66%)
city: 34 пропусков (0.01%)
sqft: 40577 пропусков (10.76%)
beds: 91282 пропусков (24.20%)
stories: 150715 пропусков (39.96%)
mls-id: 352243 пропусков (93.39%)
PrivatePool: 336874 пропусков (89.31%)
MlsId: 66880 пропусков (17.73%)
target: 2481 пропусков (0.66%)


Явные дубликаты

In [156]:

rows_before = df.shape[0] #удалим дубликаты
df = df.drop_duplicates(ignore_index=True)
rows_after = df.shape[0]
duplicates_removed = rows_before - rows_after
print(f'Удалено {duplicates_removed} дубликатов')

Удалено 50 дубликатов


Дополнительные дубликты

In [157]:
print(df['private pool'].unique())
print(df['PrivatePool'].unique())

[nan 'Yes']
[nan 'yes' 'Yes']


In [158]:

df['pool_encoded'] = df['private pool'].combine_first(df['PrivatePool'])
df = df.drop(['PrivatePool', 'private pool', 'MlsId', 'mls-id'], axis=1)
df['pool_encoded'] = df['pool_encoded'].fillna(False)
df['pool_encoded'] = df['pool_encoded'].replace(['yes', 'Yes'], True)
rows_before = df.shape[0] #удалим дубликаты
df = df.drop_duplicates(ignore_index=True)
rows_after = df.shape[0]
duplicates_removed = rows_before - rows_after
print(f'Удалено {duplicates_removed} дубликатов')

Удалено 91 дубликатов


Признак homeFacts

In [159]:
missing_values_count = df.homeFacts.isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Пропущенные значения: {missing_values_count}\n"
f"Пропущенные значения: {missing_values_percentage:.2f}%\n"
f"Уникальные значения: {df.homeFacts.nunique()}\n"
f"Уникальные значения: {list(df.homeFacts.unique()[:1])}")

Пропущенные значения: 0
Пропущенные значения: 0.00%
Уникальные значения: 321009
Уникальные значения: ["{'atAGlanceFacts': [{'factValue': '2019', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': 'Central A/C, Heat Pump', 'factLabel': 'Heating'}, {'factValue': '', 'factLabel': 'Cooling'}, {'factValue': '', 'factLabel': 'Parking'}, {'factValue': None, 'factLabel': 'lotsize'}, {'factValue': '$144', 'factLabel': 'Price/sqft'}]}"]


In [160]:
label = df.homeFacts.str.findall(r"\bfactLabel': ([\s\S]+?)[}\b]")
label[100]

["'Year built'",
 "'Remodeled year'",
 "'Heating'",
 "'Cooling'",
 "'Parking'",
 "'lotsize'",
 "'Price/sqft'"]

In [161]:
Value = df.homeFacts.str.findall(r"\bfactValue': ([\s\S]+?), 'factLabel\b")
Value[:10]

0    ['2019', '', 'Central A/C, Heat Pump', '', '',...
1    ['2019', '', '', '', '', '5828 sqft', '$159/sq...
2    ['1961', '1967', 'Forced Air', 'Central', 'Att...
3    ['2006', '2006', 'Forced Air', 'Central', 'Det...
4            ['', '', '', '', '', '10,019 sqft', None]
5    ['1920', '', 'Forced Air', 'Central', '', '680...
6    ['2006', '2006', 'Electric, Heat Pump', 'Centr...
7    ['1976', '', '', '', '', '8,750 Sq. Ft.', '$57...
8    ['1970', '', 'Forced Air', 'Central', '', '124...
9    ['2019', None, 'Gas', 'Central', 'Attached Gar...
Name: homeFacts, dtype: object

In [162]:
list_label = ','.join(label[0]).replace("'","").split(',')
list_label

['Year built',
 'Remodeled year',
 'Heating',
 'Cooling',
 'Parking',
 'lotsize',
 'Price/sqft']

In [163]:
for i, val in enumerate(list_label):
    df[val]=Value.apply(lambda x: x[i])

In [164]:
# удаляем признак homeFacts 
df = df.drop('homeFacts', axis=1)

 Признак propertyType

In [165]:
missing_values_count = df.propertyType.isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Пропущенные значения: {missing_values_count}\n"
      f"Пропущенные значения: {missing_values_percentage:.2f}%\n"
      f"Уникальные значения: {df.propertyType.nunique()}\n"
      f"Уникальные значения: {list(df.propertyType.unique()[:10])}")

Пропущенные значения: 34725
Пропущенные значения: 9.21%
Уникальные значения: 1280
Уникальные значения: ['Single Family Home', 'single-family home', 'lot/land', 'townhouse', 'Florida', nan, 'Single Family', 'coop', 'English', '2 Story']


In [166]:
df.propertyType = df.propertyType.str.lower()
missing_values_count = df.propertyType.isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Пропущенные значения: {missing_values_count}\n"
      f"Пропущенные значения: {missing_values_percentage:.2f}%\n"
      f"Уникальные значения: {df.propertyType.nunique()}\n"
      f"Уникальные значения: {list(df.propertyType.unique()[:10])}")

Пропущенные значения: 34725
Пропущенные значения: 9.21%
Уникальные значения: 1272
Уникальные значения: ['single family home', 'single-family home', 'lot/land', 'townhouse', 'florida', nan, 'single family', 'coop', 'english', '2 story']


In [167]:
df.propertyType = df.propertyType \
.str.replace('single-family home','single family') \
.str.replace('single family home','single family') \
.str.replace('yes','Other') \
.str.replace('unknown','Other')
df['Type'] = df['propertyType'].str.split(',').str[0]

In [168]:
missing_values_count = df.Type.isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Пропущенные значения: {missing_values_count}\n"
      f"Пропущенные значения: {missing_values_percentage:.2f}%\n"
      f"Уникальные значения: {df.Type.nunique()}\n"
      f"Уникальные значения: {list(df.Type.unique()[:10])}")

Пропущенные значения: 34725
Пропущенные значения: 9.21%
Уникальные значения: 256
Уникальные значения: ['single family', 'lot/land', 'townhouse', 'florida', nan, 'coop', 'english', '2 story', 'multi-family', 'penthouse']


In [169]:
synonyms_mapping = {
    'single_family_home': [
        'single family', '1 story', '2 story', 'detached', '1 story/ranch', '1 story traditional', 'single detached', 'singlefamilyresidence', 'single wide', 'single-wide mobile with land',
        'two story', 'one story', 'one level unit', 'rancher', '1 1/2 story', 'single wide mh', 'one story traditional'],
    'multi_family_home': [
        'multi-family', 'multi-family home', 'duplex', 'triplex', 'fourplex', 'multi_level', 'multi family', '2 unit condo', '2-story'],
    'condo': [
        'condo', 'coop', 'cooperative', 'condo/townhome/row home/co-op', 'condo/townhome', 'condominium', 'condo/unit', 'apartment/condo/townhouse', 'co-op', '2 story condo', 'high rise'],
    'townhouse': [
        'townhouse', 'townhome style', 'townhouse-interior', 'townhouse-end unit'],
    'apartment': [
        'apartment', 'condominium (single level)', 'high-rise', 'mid-rise', 'low-rise (1-3 stories)', 'Flats', 'studio'],
    'land': [
        'lot/land', 'land'],
    'mobile_home': [
        'mobile/manufactured', 'mobile / manufactured', 'manufactured house', 'mfd/mobile home', 'mobile home', 'manufactured home', 'manufactured double-wide', 'manufactured single-wide', 'mobile home 1 story', 'mobile manu - double wide'],
    'miscellaneous': [
        'miscellaneous'],
    'ranch': [
        'ranch', 'one story'],
    'modern': [
        'contemporary', 'contemporary/modern', "modern", 'mid-century modern', 'modern farmhouse', 'modernist'],
    'historical': [
        'historical', 'designated historical home', 'historical/conservation district', 'historic/older', 'historic vintage', 'historic'],
    'other': [
        'Other', 'english', 'urban contemporary', 'other style', 'florida', 'farms/ranches', 'carriage house', 'country english', 'straight thru', 'less than 4 floors', 'bungalow', 'custom', 'arts & crafts', 'tudor', 'new build 2019', 'split foyer', 'cottage', 'cottage/camp', 'garden home', 'farm/ranch', 'farm/ranch house', 'farm house', 'hi ranch', 'attached duplex', 'farmhouse', 'houseboat', 'ground floor', 'victorian', '3 story', '3+ stories', 'santa barbara/tuscan', 'old style', 'modular/prefab', 'post and beam', 'manuf/mobile', 'multiple occupancy', 'attached', 'hawaiian plantation', 'forest garden home', '1 1/2 story with basement', 'split-entry', 'texas hill country', 'lake house', '1 story with basement', 'hi-rise', 'coastal beach home', 'historical', 'key west/coastal', 'loft/balcony', 'english manor', 'mid-rise (4-7 stories)', 'mid-level', 'new englander', 'residential (<1 acre)', 'ranch','residential (1+ acre)', 'split', 'split level', 'split (4 level)', 'split (5+ level)', 'urban', 'patio', 'patio home', 'penthouse', 'manor', 'victorian/federal', 'coastal', 'coastal contemporary', 'coastal ii', 'coastal modern', 'coastal two story', 'mountain contemporary', 'key west', 'high ranch', 'end unit']
}


def replace_synonyms(value):
    for key, synonym_values in synonyms_mapping.items():
        if value in synonym_values:
            return key
    return "other"

In [170]:
df['Type'] = df['Type'].apply(replace_synonyms)
missing_values_count = df.Type.isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Пропущенные значения: {missing_values_count}\n"
      f"Пропущенные значения: {missing_values_percentage:.2f}%\n"
      f"Уникальные значения: {df.Type.nunique()}\n"
      f"Уникальные значения: {list(df.Type.unique()[:10])}")

Пропущенные значения: 0
Пропущенные значения: 0.00%
Уникальные значения: 12
Уникальные значения: ['single_family_home', 'land', 'townhouse', 'other', 'condo', 'multi_family_home', 'mobile_home', 'apartment', 'modern', 'miscellaneous']


In [171]:
# удаляем propertyType 
df.drop('propertyType', axis=1, inplace=True)

In [172]:
df1=df.copy()

Признак Remodeled year

In [173]:
df1['Remodeled year'] = df1['Remodeled year'].str.replace("'",'', regex=True)
mask= df1['Remodeled year'].isin(['', '0', '1111'])
df1.loc[mask,'Remodeled year'] = 'None' #уникальные значения
display(df1['Remodeled year'].sort_values().unique()[:10])

array(['1738', '1800', '1845', '1846', '1853', '1862', '1869', '1870',
       '1874', '1876'], dtype=object)

In [174]:
missing_values_count = df1['Remodeled year'].isna().sum()
total_count = len(df1)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Пропущенные значения: {missing_values_count}\n"
f"Пропущенные значения: {missing_values_percentage:.2f}%\n"
f"Уникальные значения: {df1['Remodeled year'].nunique()}\n"
f"Уникальные значения: {list(df1['Remodeled year'].sort_values().unique()[:10])}")

Пропущенные значения: 0
Пропущенные значения: 0.00%
Уникальные значения: 152
Уникальные значения: ['1738', '1800', '1845', '1846', '1853', '1862', '1869', '1870', '1874', '1876']


In [175]:
#None
print(f"Процент None: {round(df1[df1['Remodeled year']=='None'].shape[0]/df.shape[0]*100, 2)}%")
print(f"Количество None: {df1.loc[df1['Remodeled year']=='None', 'Remodeled year'].count()}")

Процент None: 60.0%
Количество None: 226242


In [176]:
# удаляем 'Remodeled year'
df1 = df1.drop('Remodeled year', axis=1)

признак Cooling

In [177]:
df1['Cooling'].str.lower().value_counts().head(10)

'central'             158742
''                    120308
'central air'          14384
'no data'              10615
'has cooling'           9730
'none'                  7390
'central electric'      6154
'wall'                  4017
'central gas'           3573
none                    3533
Name: Cooling, dtype: int64

In [178]:
df1['Cooling'] = df1['Cooling'].str.replace("'",'', regex=True)
df1['Cooling_encoded'] = df1['Cooling'].apply(lambda x: True if x not in ['', 'no data', 'None', 'none'] else False)

In [179]:
# удаляем признак 'Cooling'
df1= df1.drop('Cooling', axis=1)

Признак Heating

In [180]:
display(df1['Heating'].str.lower().value_counts().head(10))

'forced air'          134307
''                    105672
'other'                29622
'electric'             10216
'gas'                   9295
'heat pump'             8851
'no data'               8610
'central air'           7814
'central electric'      7112
'central'               6247
Name: Heating, dtype: int64

In [181]:
df1['Heating'] = df1['Heating'].str.replace("'",'', regex=True)
df1['Heating_encoded'] = df1['Heating'].apply(lambda x: True if x not in ['', 'no data', 'None', 'none'] else False)

In [182]:
missing_values_count = df1['Heating_encoded'].isna().sum()
total_count = len(df1)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Пропущенные значения: {missing_values_count}\n"
f"Пропущенные значения:: {missing_values_percentage:.2f}%\n"
f"Уникальные значения: {df1['Heating_encoded'].nunique()}\n"
f"Уникальные значения: {list(df1['Heating_encoded'].sort_values().unique())}")

Пропущенные значения: 0
Пропущенные значения:: 0.00%
Уникальные значения: 2
Уникальные значения: [False, True]


In [183]:
# удаляем признак 'Heating'
df1= df1.drop('Heating', axis=1)

Признак city

In [184]:
df1['city'] = df1['city'].str.title()
missing_values_count = df1['city'].isna().sum()
total_count = len(df1)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Пропущенные значения: {missing_values_count}\n"
f"Пропущенные значения: {missing_values_percentage:.2f}%\n"
f"Уникальные значения: {df1['city'].nunique()}\n"
f"Уникальные значения: {list(df1['city'].sort_values().unique()[:10])}")

Пропущенные значения: 32
Пропущенные значения: 0.01%
Уникальные значения: 1909
Уникальные значения: [' ', '--', 'Abilene', 'Abingdon', 'Accokeek', 'Adams', 'Addison', 'Adel', 'Adelphi', 'Advance']


In [185]:
df1['city'].str.lower().value_counts().sort_index().head(10)

             25
--            9
abilene     134
abingdon    116
accokeek      3
adams         5
addison      25
adel         13
adelphi       6
advance       3
Name: city, dtype: int64

In [186]:

df1['city'] = df1['city'].str.replace(' City', '') 
initial_row_count = df1.shape[0]
df1 = df1[~df1['city'].isin([' ', '--', np.nan])]
removed_row_count = initial_row_count - df1.shape[0]
print(f"Удаленные строки: {removed_row_count}")


Удаленные строки: 66


In [187]:
missing_values_count = df1['city'].isna().sum()
total_count = len(df1)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Пропущенные значения: {missing_values_count}\n"
f"Пропущенные значения: {missing_values_percentage:.2f}%\n"
f"Уникальные значения: {df1['city'].nunique()}\n"
f"Уникальные значения: {list(df1['city'].sort_values().unique()[:10])}")

Пропущенные значения: 0
Пропущенные значения: 0.00%
Уникальные значения: 1897
Уникальные значения: ['Abilene', 'Abingdon', 'Accokeek', 'Adams', 'Addison', 'Adel', 'Adelphi', 'Advance', 'Akron', 'Alamo Heights']


Признак sqft

In [188]:
missing_values_count = df1['sqft'].isna().sum()
total_count = len(df1)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Пропущенные значения: {missing_values_count}\n"
f"Пропущенные значения: {missing_values_percentage:.2f}%\n"
f"Уникальные значения: {df1['sqft'].nunique()}\n"
f"Уникальные значения: {list(df1['sqft'].sort_values().unique()[:10])}")

Пропущенные значения: 40517
Пропущенные значения: 10.75%
Уникальные значения: 25396
Уникальные значения: ['--', '-- sqft', '0', '1', '1 sqft', '1,000', '1,000 sqft', '1,001', '1,001 sqft', '1,002']


In [189]:
s_sqft = df1.sqft[df1.sqft.notna()]
s_sqft = s_sqft.str.findall(r"\d+")
s_sqft = s_sqft.apply(lambda x: ''.join(x))
df1.loc[df1.sqft.notna(), 'sqft']=s_sqft
df1.sqft = df1.sqft.fillna(0)
df1.loc[df1.sqft=='','sqft']=0
df1.sqft = df1.sqft.astype(int)

In [190]:
selected_rows = df1[(df1['sqft'] == 0) & ((df1['Type'] == 'other') | (df1['Type'] == 'land') | (df1['Type'] == 'ranch'))]
percent_selected = len(selected_rows) / len(df1) * 100
# Выводим полученные строки и процент выбранных строк
print(f"Выбранные строки: {len(selected_rows)}")
print(f"Выбранные строки: {percent_selected:.2f}%")

Выбранные строки: 46146
Выбранные строки: 12.24%


In [191]:
selected_rows = df1[(df1['sqft'] == 0) & ~(df1['Type'].isin(['other', 'land', 'ranch']))]
unique_statuses = selected_rows['Type'].unique()
# Выводим полученные строки и процент выбранных строк
percent_selected = len(selected_rows) / len(df1) * 100
print(f"Выбранные строки: {len(selected_rows)}")
print(f"Выбранные строки: {percent_selected:.2f}%")

Выбранные строки: 6981
Выбранные строки: 1.85%


Признак baths

In [192]:
missing_values_count = df1.baths.isna().sum()
total_count = len(df1)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Пропущенные значения: {missing_values_count}\n"
      f"Пропущенные значения: {missing_values_percentage:.2f}%\n"
      f"Уникальные значения: {df1.baths.nunique()}\n"
      f"Уникальные значения: {list(df1.baths.unique()[:10])}")

Пропущенные значения: 106210
Пропущенные значения: 28.17%
Уникальные значения: 229
Уникальные значения: ['3.5', '3 Baths', '2 Baths', '8 Baths', nan, '2', '3', 'Bathrooms: 2', '1,750', '4 Baths']


In [193]:
df1.baths = df1.baths.str.replace('[a-zA-Z:]','', regex=True)
df1.baths = df1.baths.str.replace(' ','', regex=True)

In [194]:
df1['baths'] = df1['baths'].str.replace('[a-zA-Z+:]','', regex=True)
df1['baths'].replace(['nan','', '~', '..', '--', '—','0.0','0.00','0/0'], 0, inplace=True)
df1['baths'].replace(['1-2','1/1-0/1-0/1-0', '1/1/1/1','1-0/1-0/1', ], 1, inplace=True)
df1['baths'].replace(['2-1/2-1/1-1/1-1','3-1/2-2',], 2, inplace=True)
df1['baths'].replace(['116/116/116'], 116, inplace=True)
df1['baths'].replace(['7,500'], 7.5, inplace=True)
df1['baths'].replace(['5,000'], 5, inplace=True)
df1['baths'].replace(['3,500'], 3.5, inplace=True)
df1['baths'].replace(['2,750'], 2.75, inplace=True)
df1['baths'].replace(['2,250'], 2.25, inplace=True)
df1['baths'].replace(['1,250'], 1.25, inplace=True)
df1['baths'].replace(['1,500'], 1.5, inplace=True)
df1['baths'].replace(['4,000'], 4, inplace=True)
df1['baths'].replace(['2,000'], 2, inplace=True)
df1['baths'].replace(['3,000'], 3, inplace=True)
df1['baths'].replace(['2,500'], 2.5, inplace=True)
df1['baths'].replace(['1,000'], 1, inplace=True)
df1['baths'].replace(['1,750'], 1.75, inplace=True)
df1['baths'] = df1['baths'].fillna(0)

In [195]:
df1['baths'] = df1['baths'].astype(float).round()
selected_rows = df1[(df1['baths'] == 0) & ((df1['Type'] == 'other') | (df1['Type'] == 'land') | (df1['Type'] == 'ranch'))]
display(selected_rows.head())
percent_selected = len(selected_rows) / len(df1) * 100

print(f"Выбранные строки: {percent_selected:.2f}%")

,status,street,baths,fireplace,city,schools,sqft,zipcode,beds,state,stories,target,pool_encoded,Year built,Parking,lotsize,Price/sqft,Type,Cooling_encoded,Heating_encoded
4,for sale,1524 Kiscoe St,0.0,NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",0,32908,NaN,FL,NaN,"$5,000",False,'','',"'10,019 sqft'",None,land,False,False
6,Active,552 Casanova Ct,0.0,NaN,Poinciana,"[{'rating': ['3', '3', '1', 'NR'], 'data': {'D...",1507,34759,NaN,FL,One,"181,500",False,'2006','',"'4,996 Sq. Ft.'",'$120 / Sq. Ft.',other,True,True
7,Active,6094 Mingle Dr,0.0,NaN,Memphis,"[{'rating': ['4', '2', '2'], 'data': {'Distanc...",0,38115,NaN,TN,NaN,"68,000",False,'1976','',"'8,750 Sq. Ft.'",'$57 / Sq. Ft.',other,False,False
11,Active,4222 False Cypress Ln,0.0,NaN,Houston,"[{'rating': ['6', '3', '6', '1', '5'], 'data':...",3130,77068,NaN,TX,2,"260,000",False,'2015','',"'5,715 Sq. Ft.'",'$83 / Sq. Ft.',other,True,True
40,Active,1104 Rotonda Cir,0.0,NaN,Rotonda West,"[{'rating': ['6', '6', '7'], 'data': {'Distanc...",0,33947,NaN,FL,NaN,"54,000",False,'','','—',None,other,False,False


Выбранные строки: 19.35%


In [196]:
selected_rows = df1[(df1['baths'] == 0) & ~(df1['Type'].isin(['other', 'land', 'ranch']))]
unique_statuses = selected_rows['Type'].unique()
print(f"Уникальные значения признака Type: {unique_statuses}")
percent_selected = len(selected_rows) / len(df1) * 100
print(f"Выбранные строки: {percent_selected:.2f}%")


Уникальные значения признака Type: ['townhouse' 'single_family_home' 'condo' 'apartment' 'modern'
 'miscellaneous' 'multi_family_home' 'mobile_home' 'historical']
Выбранные строки: 10.26%


In [197]:
missing_values_count = df1.baths.isna().sum()
total_count = len(df1)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Пропущенные значения: {missing_values_count}\n"
      f"Пропущенные значения: {missing_values_percentage:.2f}%\n"
      f"Уникальные значения: {df1.baths.nunique()}\n"
      f"Уникальные значения: {list(df1.baths.unique()[:10])}")

Пропущенные значения: 0
Пропущенные значения: 0.00%
Уникальные значения: 49
Уникальные значения: [4.0, 3.0, 2.0, 8.0, 0.0, 5.0, 1.0, 7.0, 6.0, 9.0]


Признак zipcode

In [198]:
df2=df1.copy()

In [199]:
missing_values_count = df2['zipcode'].isna().sum()
total_count = len(df2)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Пропущенные значения: {missing_values_count}\n"
f"Пропущенные значения: {missing_values_percentage:.2f}%\n"
f"Уникальные значения: {df2['zipcode'].nunique()}\n"
f"Уникальные значения: {list(df2['zipcode'].sort_values().unique()[:10])}")

Пропущенные значения: 0
Пропущенные значения: 0.00%
Уникальные значения: 4549
Уникальные значения: ['--', '0', '00000', '02108', '02109', '02110', '02111', '02113', '02114', '02115']


In [200]:
df2[(df2['zipcode']=='--') | (df2['zipcode']=='0')| (df2['zipcode']=='00000') ]

,status,street,baths,fireplace,city,schools,sqft,zipcode,beds,state,stories,target,pool_encoded,Year built,Parking,lotsize,Price/sqft,Type,Cooling_encoded,Heating_encoded
30258,Active,Gates Canyon Rd,0.0,NaN,Vacaville,"[{'rating': ['7', '4', '6', '6', '10', '2'], '...",0,0,NaN,CA,NaN,"$380,000",False,'','','222156',None,land,False,False
83514,New,Cornejo Ricardo Descalzi,0.0,NaN,Quito Ecuador,"[{'rating': [], 'data': {'Distance': [], 'Grad...",0,0,NaN,NY,NaN,"470,000",False,'1995','','—',None,other,True,False
231230,New construction,0 N Gopher Canyon Rd,0.0,NaN,Bonsall,"[{'rating': ['7/10', '7/10', '3/10'], 'data': ...",0,--,449 acres,CA,NaN,"$60,000,000",False,'','','','',other,False,False
235154,NaN,1744 N Dixie Hwy # 1744,3.0,NaN,Fort Lauderdale,"[{'rating': ['3/10', '5/10', '7/10'], 'data': ...",2043,--,3,FL,NaN,"$425,000",False,'2010','No Data','No Data','$208',townhouse,True,True
305481,for sale,000 U.S. Hwy 359,0.0,NaN,Laredo,"[{'rating': ['4/10'], 'data': {'Distance': ['7...",243849,0,NaN,TX,NaN,"$1,740,000",False,'','','58.00 acres','$7/sqft',land,False,False
308135,Active,435 pescador,0.0,NaN,Foreign Country,"[{'rating': [], 'data': {'Distance': [], 'Grad...",1100,00000,NaN,OS,NaN,"125,000",False,'1972','','—','$114 / Sq. Ft.',other,False,False


In [201]:
indexes_to_drop = df2[(df2['zipcode']=='--') | (df2['zipcode']=='0')| (df2['zipcode']=='00000')].index
df2 = df2.drop(indexes_to_drop)
df2.zipcode.sort_values().unique()[:10]

array(['02108', '02109', '02110', '02111', '02113', '02114', '02115',
       '02116', '02118', '02119'], dtype=object)

In [202]:
df2.zipcode = df2.zipcode.str.replace("-.+",'', regex=True)
df2.zipcode.sort_values().unique()[:10]

array(['02108', '02109', '02110', '02111', '02113', '02114', '02115',
       '02116', '02118', '02119'], dtype=object)

Признак beds

In [203]:
missing_values_count = df2['beds'].isna().sum()
total_count = len(df2)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Пропущенные значения: {missing_values_count}\n"
f"Пропущенные значения: {missing_values_percentage:.2f}%\n"
f"Уникальные значения: {df2['beds'].nunique()}\n"
f"Уникальные значения: {list(df2['beds'].sort_values().unique()[:10])}")

Пропущенные значения: 91156
Пропущенные значения: 24.18%
Уникальные значения: 1181
Уникальные значения: [' ', '# Bedrooms 1st Floor', '-- bd', '-- sqft', '0', '0.0', '0.25 acres', '0.26 acres', '0.27 acres', '0.28 acres']


In [204]:
# удаляем признак Beds
df2 = df2.drop('beds', axis=1)

Признак stories

In [205]:
missing_values_count = df2['stories'].isna().sum()
total_count = len(df2)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Пропущенные значения: {missing_values_count}\n"
f"Пропущенные значения: {missing_values_percentage:.2f}%\n"
f"Уникальные значения: {df2['stories'].nunique()}\n"
f"Уникальные значения: {list(df2['stories'].sort_values().unique()[:10])}")

Пропущенные значения: 150516
Пропущенные значения: 39.93%
Уникальные значения: 347
Уникальные значения: [', 1', ', 2', ', 3', '0', '0.0', '1', '1 1/2 Levels', '1 1/2 Story', '1 Level', '1 Level, 1.5 Level']


In [206]:
df2[df2.stories.isna()].Type.value_counts()

single_family_home    46300
other                 36999
land                  28817
condo                 24355
multi_family_home      5212
townhouse              3937
mobile_home            1499
ranch                  1389
modern                 1129
apartment               824
historical               44
miscellaneous            11
Name: Type, dtype: int64

In [207]:
# удаляем признак Beds
df2 = df2.drop('stories', axis=1)

Признак Target

In [208]:
missing_values_count = df2['target'].isna().sum()
total_count = len(df2)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Пропущенные значения: {missing_values_count}\n"
f"Пропущенные значения: {missing_values_percentage:.2f}%\n"
f"Уникальные значения: {df2['target'].nunique()}\n"
f"Уникальные значения: {list(df2['target'].sort_values().unique()[:10])}")

Пропущенные значения: 2475
Пропущенные значения: 0.66%
Уникальные значения: 43937
Уникальные значения: ['$1', '$1,000', '$1,000,000', '$1,000,000+', '$1,000,036', '$1,000,050', '$1,000,100', '$1,000,898', '$1,000/mo', '$1,001,713']


In [209]:
df2 = df2.dropna(subset=['target'])
df2.loc[:,'target'] = df2.loc[:,'target'].str.replace('[^0-9]','',regex=True)
df2.target = df2.target.astype(int)
display(df2[df2['target'] == df2['target']. min ()].head())
display(df2[df2['target'] == df2['target']. max ()].head())

,status,street,baths,fireplace,city,schools,sqft,zipcode,state,target,pool_encoded,Year built,Parking,lotsize,Price/sqft,Type,Cooling_encoded,Heating_encoded
15533,Active,1230 Old Dickerson Rd,2.0,NaN,Goodlettsville,"[{'rating': ['3', '2', '3', '3', '2', '1', 'NR...",1596,37072,TN,1,False,'1940',"'Detached, ASPHT'",'87120','No Info',single_family_home,False,True
25334,NaN,3205 N High School Rd,1.0,Yes,Indianapolis,"[{'rating': ['2/10', 'NA', '1/10'], 'data': {'...",1586,46224,IN,1,False,'1957','Detached Garage','0.54 acres',None,single_family_home,True,True
84943,foreclosure,7804 Hamilton Ave,0.0,yes,Cincinnati,"[{'rating': ['2/10'], 'data': {'Distance': ['0...",2536,45231,OH,1,False,'1865','Detached Garage','0.31 acres','$0/sqft',land,True,False
148219,Active,Cummings St,0.0,NaN,Huntley,"[{'rating': [], 'data': {'Distance': [], 'Grad...",0,60142,IL,1,False,'','','1558141',None,land,False,False
205315,Active,Route 47 Hwy,0.0,NaN,Huntley,"[{'rating': ['8', '9', '8', '7', '7', 'NR'], '...",0,60142,IL,1,False,'','','671260',None,land,False,False


,status,street,baths,fireplace,city,schools,sqft,zipcode,state,target,pool_encoded,Year built,Parking,lotsize,Price/sqft,Type,Cooling_encoded,Heating_encoded
132407,For sale,875 Nimes Rd,18.0,Yes,Los Angeles,"[{'rating': ['10/10', '7/10'], 'data': {'Dista...",0,90077,CA,195000000,True,'1935','40 spaces','6.49 acres','No Data',single_family_home,True,True


In [210]:
count = len(df2[df2['target'] < 900])
percentage = count / len(df2) * 100
print(f"Количество строк: {count}")
print(f"Процент строк: {percentage:.2f}%")

Количество строк: 91
Процент строк: 0.02%


In [211]:
df2 = df2.drop(df2[df2['target'] < 900].index)

In [212]:
display(df2[df2['target'] == df2['target']. min ()].head())

,status,street,baths,fireplace,city,schools,sqft,zipcode,state,target,pool_encoded,Year built,Parking,lotsize,Price/sqft,Type,Cooling_encoded,Heating_encoded
15005,NaN,3110 4th Street Cir,0.0,NaN,Jacksonville,"[{'rating': ['2/10', '2/10', '3/10'], 'data': ...",0,32254,FL,900,False,None,'No Data',None,'$1',single_family_home,True,True
82420,for rent,378 Old Grist Mill Blvd,0.0,NaN,Gray,"[{'rating': ['6/10', '8/10'], 'data': {'Distan...",804,37615,TN,900,False,'','Garage Attached','',None,single_family_home,False,False
115104,for rent,105 Alma Ave,0.0,NaN,Buffalo,"[{'rating': ['1/10', '4/10', 'None/10', '5/10'...",1400,14215,NY,900,False,'1920','','',None,multi_family_home,False,False


In [213]:
missing_data = df2.isnull().sum()
missing_columns = missing_data[missing_data > 0]

if missing_columns.empty:
    print('Пропуски отсутствуют')
else:
    for column_name in missing_columns.index:
        missing_count = missing_columns[column_name]
        percentage = (missing_count / len(df2)) * 100
        print(f'{column_name}: {missing_count} Пропуски ({percentage:.2f}%)')

status: 39238 Пропуски (10.48%)
street: 2 Пропуски (0.00%)
fireplace: 271899 Пропуски (72.62%)


признак Fireplace

In [214]:
df2['fireplace'].str.lower().value_counts().head(10)

yes               70624
1                 14532
2                  2432
not applicable     1990
fireplace           847
3                   563
living room         433
location            399
wood burning        311
gas/gas logs        300
Name: fireplace, dtype: int64

In [215]:
df2['fireplace_encoded'] = df1['fireplace'].apply(lambda x: True if x not in ['', 'no data', 'None', 'none', '0', 'not applicable', 'no'] and not pd.isna(x) else False)


In [216]:
# удаляем признак fireplace
df2.drop('fireplace', axis=1, inplace=True)

Признак Status

In [217]:
missing_values_count = df2.status.isna().sum()
total_count = len(df2)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Количество пропущенных значений: {missing_values_count}\n"
      f"Процент пропущенных значений: {missing_values_percentage:.2f}%\n"
      f"Количество уникальных значений: {df2.status.nunique()}\n"
      f"Уникальные значения: {list(df2.status.unique()[:10])}")

Количество пропущенных значений: 39238
Процент пропущенных значений: 10.48%
Количество уникальных значений: 156
Уникальные значения: ['Active', 'for sale', nan, 'New construction', 'New', 'For sale', 'Pending', 'P', 'Active/Contingent', ' / auction']


In [218]:
categories = {
    "For Sale": ["for sale", "For sale", "New construction", "New"],
    "Active": [
        "Active", "A Active", "Active/Contingent", "Active Under Contract", "Active Option", "Auction - Active",
        "Active With Contingencies", "Active Option Contract", "Active Contingency", "Active Backup",
        "Active Contingent", "Active - Auction", "Active With Offer", "Active - Contingent", "Active with Contract",
        "Temporary Active", "Re Activated", "Reactivated"
    ],
    "Pending": [
        "P", "Pending", "pending", "P Pending Sale", "Pending Ab", "Pending Continue To Show",
        "Pending Inspection", "Pending Offer Approval", "Pending In", "Pending W/Insp Finance", "Pending Fe",
        "Pending W/Backup Wanted", "Pending Backups Wanted", "Pending With Contingencies", "Lease/Purchase Pending",
        "Pending Bring Backup", "Pending - Taking Backups", "Pending - Continue to Show",
        "Pending Taking Backups", "Offer Pending Signature", "Pending (Do Not Show)", "Pending W/ Cont.",
        "Pending W/Escape Clause", "Pending - Backup Offer Requested", "Pending Sale"
    ],
    "Contingent": [
        "Contingent", "Contingent Finance And Inspection", "Contingent Show",
        "Contingent Take Backup", "Contingent - Sale of Home", "Contingent Finance and Inspection",
        "C Continue Show", "Contingent   Show", "Contingent   Release", "Contingent   No Show",
        "CT Insp - Inspection Contingency", "Contingent   Foreclosure", "Conting Accpt Backups",
        "Contingent - Financing", "Contingency 48 Hr (+/ )", "Contingency Contract", "Contingent Escape"
    ],
    "Foreclosure": [
        "Pre-foreclosure", "Pre-foreclosure / auction", " / auction", "Foreclosed", "foreclosure", "Foreclosure"
    ],
    "Under Contract": [
        "Under Contract", "Under Contract   Showing", "Under Contract Backups", "Under Contract Show",
        "Under Contract - Show", "Under Contract - No Show", "Under contract", "U Under Contract",
        "Due Diligence Period", "Contract Contingent On Buyer Sale", "Contract P", "Ct", "Uc Continue To Show",
        "Under Contract Taking Back Up Offers", "Under Contract W/ Bckp", "Contract"
    ],
    "For Rent": ["for rent", "Apartment for rent", "Condo for rent"],
    "Auction": ["Auction"],
    "Recently Sold": ["recently sold"],
    "Price Change": ["Price Change"],
    "Back on Market": ["Back on Market", "Back On Market"],
    "Closed": ["Closed"],
    "Listing Extended": ["Listing Extended"],
    "Coming Soon": [
        "Coming soon: Nov 21.", "Coming soon: Dec 4.", "Coming soon: Nov 23.", "Coming soon: Nov 29.",
        "Coming soon: Dec 2.", "Coming soon: Dec 10.", "Coming soon: Dec 24.", "Coming soon: Nov 14.",
        "Coming soon: Nov 22.", "Coming soon: Oct 21.", "Coming soon: Dec 14.", "Coming soon: Oct 24.",
        "Coming soon: Dec 18.", "Coming soon: Dec 16.", "Coming soon: Dec 3.", "Coming soon: Dec 25.",
        "Coming soon: Nov 11.", "Coming soon: Nov 28.", "Coming soon: Nov 17.", "Coming soon: Dec 6.",
        "Coming soon: Nov 27.", "Coming soon: Nov 26.", "Coming soon: Dec 7.", "Coming soon: Dec 27.",
        "Coming soon: Dec 11.", "Coming soon: Dec 5.", "Coming soon: Nov 13.", "Coming soon: Nov 19.",
        "Coming soon: Nov 8.", "Coming soon: Oct 29.", "Coming soon: Dec 15.", "Coming soon: Oct 30.",
        "Coming soon: Dec 9.", "Coming soon: Dec 20.", "Coming soon: Dec 13.", "Coming soon: Dec 23.",
        "Coming soon: Nov 30.", "Coming soon: Dec 1.", "Coming soon: Nov 5.", "Coming soon: Nov 12.",
        "Coming soon: Nov 25.", "Coming soon: Nov 9."
    ],
}

def categorize_status(status_value):
    for category, values in categories.items():
        if status_value in values:
            return category
    return "Other"

# заменим значения на категории
df2['status'] = df2['status'].apply(categorize_status)

In [219]:
# выполним проверку
missing_values_count = df2.status.isna().sum()
total_count = len(df2)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Пропущенные значения: {missing_values_count}\n"
      f"Пропущенные значения: {missing_values_percentage:.2f}%\n"
      f"Уникальные значения: {df2.status.nunique()}\n"
      f"Уникальные значения: {list(df2.status.unique()[:10])}")



Пропущенные значения: 0
Пропущенные значения: 0.00%
Уникальные значения: 14
Уникальные значения: ['Active', 'For Sale', 'Other', 'Pending', 'Foreclosure', 'Under Contract', 'For Rent', 'Auction', 'Contingent', 'Price Change']


In [220]:

rows_before = df2.shape[0]
df2= df2.drop_duplicates(ignore_index=True)
rows_after = df2.shape[0]
duplicates_removed = rows_before - rows_after
print(f'Удалено {duplicates_removed} дубликатов')

Удалено 34 дубликатов


Признак  Street

In [221]:
missing_values_count = df2.street.isna().sum()
total_count = len(df)
missing_values_percentage = (missing_values_count / total_count) * 100

print(f"Пропущенные значения: {missing_values_count}\n"
      f"Пропущенные значения: {missing_values_percentage:.2f}%\n"
      f"Уникальные значения: {df.street.nunique()}")

Пропущенные значения: 2
Пропущенные значения: 0.00%
Уникальные значения: 337076


In [222]:
# удаляем строки, где есть пропуски в столбце street
df2 = df2.dropna(subset=['street'])

In [223]:
df3=df2.copy()

In [224]:
missing_data = df3.isnull().sum()
missing_columns = missing_data[missing_data > 0]

if missing_columns.empty:
    print('Пропуски отсутствуют')
else:
    for column_name in missing_columns.index:
        missing_count = missing_columns[column_name]
        percentage = (missing_count / len(df2)) * 100
        print(f'{column_name}: {missing_count} Пропуски ({percentage:.2f}%)')

Пропуски отсутствуют


In [225]:
df3.duplicated().sum() 

0

In [226]:
# Сохраняеи данные в CSV-файл
df3.to_csv("data/cleaned1_data.csv", index=False)